In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./AT_interpolated.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./AT_interpolated.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def2/y_pred_{attack_name}{eps}_Def2.npy", all_preds)

        

In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Adversarial training function with interpolation
def interpolated_adversarial_training(model, train_loader, optimizer, epoch, device, eps=0.3, alpha=2/255, iters=40):
    model.train()
    loss_fn = nn.CrossEntropyLoss()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Create adversarial examples
        data_adv = data.clone().detach().requires_grad_(True).to(device)
        outputs = model(data_adv)
        loss = loss_fn(outputs, target)
        loss.backward()
        data_adv = data_adv + alpha * data_adv.grad.sign()
        eta = torch.clamp(data_adv - data, min=-eps, max=eps)
        data_adv = torch.clamp(data + eta, min=0, max=1).detach_()

        # Interpolation
        lam = np.random.beta(1.0, 1.0)
        data_mixed = lam * data + (1 - lam) * data_adv
        target_mixed = lam * target + (1 - lam) * target

        optimizer.zero_grad()
        output = model(data_mixed)
        target_mixed = target_mixed.long()
        
        loss = loss_fn(output, target_mixed)
        loss.backward()
        optimizer.step()

        if batch_idx % 1000 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Model initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = input_shape  # Replace with your input shape
output_size = output_shape  # Replace with your output shape
model = DNNModel(input_size=input_size, output_size=output_size).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')



In [11]:
# # Training loop with early stopping and validation evaluation
# for epoch in range(10):
#     interpolated_adversarial_training(model, train_loader, optimizer, epoch, device, eps=0.3, alpha=2/255, iters=40)

#     # Validation
#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break


In [12]:
model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adversarial_Training/Adversarial_Training_interpolated.pt"))


/tmp/ipykernel_3493/413485849.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adv

<All keys matched successfully>

In [13]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./AT_interpolated.csv", mode='a', index=False, header=False)

In [15]:
# percentage = ["100", "50", "20", "1"]
# model_name = ["XGB", "RF", "DT" ]

percentage = ["10"]
model_name = ["RF"]

In [16]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start RF 10 baseline
(97, 56) (97,)
Accuracy: 0.9278350515463918

macro
Precision: 0.9213098729227762
Recall: 0.914423076923077
F1 Score: 0.917726887192536

weighted
Precision: 0.9274621841964709
Recall: 0.9278350515463918
F1 Score: 0.9275377525948076

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_BIM_Def2.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_FGSM_Def2.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_PGD_Def2.npy not found
start RF 10 DF
(402, 56) (402,)
Accuracy: 0.9577114427860697

macro
Precision: 0.4961340206185567
Recall: 0.4824561403508772
F1 Score: 0.4891994917407878

weighted
Precision: 0.9848630558547469
Recall: 0.9577114427860697
F1 Score: 0.9710974985302206

start RF 10 AutoPGD
(186, 56) (186,)
Accuracy: 0.967741935483871

macro
Precision: 0.9

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start RF 10 baseline
(104, 56) (104,)
Accuracy: 0.8461538461538461

macro
Precision: 0.8389610389610389
Recall: 0.8684953610326744
F1 Score: 0.8414029736942432

weighted
Precision: 0.874925074925075
Recall: 0.8461538461538461
F1 Score: 0.8493210944602481

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_BIM_Def2.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_FGSM_Def2.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_PGD_Def2.npy not found
start RF 10 DF
(402, 56) (402,)
Accuracy: 0.9800995024875622

macro
Precision: 0.5949622166246852
Recall: 0.5949622166246852
F1 Score: 0.5949622166246852

weighted
Precision: 0.9800995024875622
Recall: 0.9800995024875622
F1 Score: 0.9800995024875622

start RF 10 AutoPGD
(183, 56) (183,)
Accuracy: 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
(89, 56) (89,)
Accuracy: 0.9101123595505618

macro
Precision: 0.9
Recall: 0.9298245614035088
F1 Score: 0.9067085953878407

weighted
Precision: 0.9280898876404494
Recall: 0.9101123595505618
F1 Score: 0.9117141309212541

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_BIM_Def2.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_FGSM_Def2.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_PGD_Def2.npy not found
start RF 10 DF
(330, 56) (330,)
Accuracy: 0.9606060606060606

macro
Precision: 0.4953125
Recall: 0.4847094801223242
F1 Score: 0.4899536321483771

weighted
Precision: 0.9816193181818181
Recall: 0.9606060606060606
F1 Score: 0.970999016439511

start RF 10 AutoPGD
(180, 56) (180,)
Accuracy: 0.933333333333333

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.08133971291866028

macro
Precision: 0.5
Recall: 0.04066985645933014
F1 Score: 0.0752212389380531

weighted
Precision: 1.0
Recall: 0.08133971291866028
F1 Score: 0.1504424778761062

start RF 10 FGSM
(27, 56) (27,)
Accuracy: 0.8518518518518519

macro
Precision: 0.4791666666666667
Recall: 0.4423076923076923
F1 Score: 0.46

weighted
Precision: 0.9228395061728395
Recall: 0.8518518518518519
F1 Score: 0.885925925925926

start RF 10 PGD
(418, 56) (418,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.08133971291866028

macro
Precision: 0.5
Recall: 0.04066985645933014
F1 Score: 0.0752212389380531

weighted
Precision: 1.0
Recall: 0.08133971291866028
F1 Score: 0.1504424778761062

start RF 10 DF
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 AutoPGD
(484, 56) (484,)
Accuracy: 0.365702479338843

macro
Precision: 0.5766091777024946
Recall: 0.586424042386867
F1 Score: 0.3650926586648777

weighted
Precision: 0.8101130223193946
Recall: 0.365702479338843
F1 Score: 0.3776956192601595

start RF 10 ZOO
(252, 56) (252,)
Accuracy: 0.9047619047619048

macro
Precision: 0.8894830659536542
Recall: 0.9109717868338558
F1 Score: 0.8977480219111382

weighted
Precision: 0.911552499787794
Recall: 0.9047619047619048
F1 Score: 0.9060371561893169

start RF 10 CaFA
(6, 56) (6,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 SINIFG

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9711538461538461

macro
Precision: 0.9730620878437228
Recall: 0.969924192485168
F1 Score: 0.970996229923049

weighted
Precision: 0.9720443589424551
Recall: 0.9711538461538461
F1 Score: 0.9711058759966471

start BIM
Accuracy: 0.6095505617977528

macro
Precision: 0.5943458980044346
Recall: 0.5824745598449362
F1 Score: 0.5796144795303753

weighted
Precision: 0.6003830439223699
Recall: 0.6095505617977528
F1 Score: 0.5966307789244635

start FGSM
Accuracy: 0.6520547945205479

macro
Precision: 0.6462015503875969
Recall: 0.6435574229691876
F1 Score: 0.6442686230421537

weighted
Precision: 0.6500736965063183
Recall: 0.6520547945205479
F1 Score: 0.6504687225527269

start PGD
Accuracy: 0.6095505617977528

macro
Precision: 0.5943458980044346
Recall: 0.5824745598449362
F1 Score: 0.5796144795303753

weighted
Precision: 0.6003830439223699
Recall: 0.6095505617977528
F1 Score: 0.5966307789244635

start DF
Accuracy: 0.8621190130624092

macro
Precision: 0.5152391975308642
Recal

In [19]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start RF 10 baseline
(770, 56) (770,)
Accuracy: 0.9766233766233766

macro
Precision: 0.55
Recall: 0.98828125
F1 Score: 0.5849802371541502

weighted
Precision: 0.9976623376623377
Recall: 0.9766233766233766
F1 Score: 0.9860479441507108

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input10/x_test_adv_BIM_Def2.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input10/x_test_adv_FGSM_Def2.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input10/x_test_adv_PGD_Def2.npy not found
start RF 10 DF
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input10/x_test_adv_DF_Def2.npy not found
start RF 10 AutoPGD
(75, 56) (75,)
Accuracy: 0.72

macro
Precision: 0.6111111111111112
Recall: 0.8478260869565217
F1 Score: 0.592074592074592

weighted
Precision: 0.9377777777777777
Recall:

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

In [20]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start RF 10 baseline
(40, 56) (40,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 BIM
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 FGSM
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 PGD
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 DF
(141, 56) (141,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 AutoPGD
(120, 56) (120,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 ZOO
(122, 56) (122,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Rec

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]


active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
Accuracy: 0.7139945652173914

macro
Precision: 0.6088986684721283
Recall: 0.6355668872125528
F1 Score: 0.6155057921190451

weighted
Precision: 0.7531545012657856
Recall: 0.7139945652173914
F1 Score: 0.7294619295831718

start RF 10 BIM
Accuracy: 0.5558510638297872

macro
Precision: 0.49902790279027903
Recall: 0.4991874322860238
F1 Score: 0.49233144954603153

weighted
Precision: 0.5290723327651914
Recall: 0.5558510638297872
F1 Score: 0.5362698293513363

start RF 10 FGSM
Accuracy: 0.5774456521739131

macro
Precision: 0.5043520714186078
Recall: 0.5038670015864621
F1 Score: 0.501670922893712

weighted
Precision: 0.5568744048954739
Recall: 0.5774456521739131
F1 Score: 0.565036550166354

start RF 10 PGD
Accuracy: 0.5558510638297872

macro
Precision: 0.49902790279027903
Recall: 0.4991874322860238
F1 Score: 0.49233144954603153

weighted
Precision: 0.5290723327651914
Recall: 0.5558510638297872
F1 Score: 0.5362698293513363

start RF 10 DF
Accuracy: 0.7777777777777778

macro
P

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.9444444444444444

macro
Precision: 0.4938837920489297
Recall: 0.47781065088757396
F1 Score: 0.4857142857142857

weighted
Precision: 0.9762147468569488
Recall: 0.9444444444444444
F1 Score: 0.9600668337510443

start RF 10 AutoPGD
Accuracy: 0.9918533604887984

macro
Precision: 0.9937888198757764
Recall: 0.9884393063583815
F1 Score: 0.9910270467836257

weighted
Precision: 0.9919545609796208
Recall: 0.9918533604887984
F1 Score: 0.9918311775705386

start RF 10 ZOO
Accuracy: 0.9856230031948882

macro
Precision: 0.9857975460122699
Recall: 0.9854550470526928
F1 Score: 0.9856035692109559

weighted
Precision: 0.9856632823065917
Recall: 0.9856230031948882
F1 Score: 0.9856204683274187

start RF 10 CaFA
Accuracy: 0.8712121212121212

macro
Precision: 0.7954545454545454
Recall: 0.7672897196261682
F1 Score: 0.7799784292577704

weighted
Precision: 0.8660468319559228
Recall: 0.8712121212121212
F1 Score: 0.8679921085549088

start RF 10 SINIFGSM
Accuracy: 0.022727272727272728

macro
Precision: 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

In [18]:
# torch.save(model.state_dict(), "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adversarial_Training/Adversarial_Training_interpolated.pt")

In [ ]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")